# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 8 2022 1:48PM,245983,16,EMERSA-8501475,"Emersa Waterbox, LLC",Released
1,Sep 8 2022 1:47PM,245982,10,0085969863,ISDIN Corporation,Released
2,Sep 8 2022 1:47PM,245982,10,0085969864,ISDIN Corporation,Released
3,Sep 8 2022 1:47PM,245982,10,0085969862,ISDIN Corporation,Released
4,Sep 8 2022 1:47PM,245982,10,0085969902,ISDIN Corporation,Released
5,Sep 8 2022 1:47PM,245982,10,0085969906,ISDIN Corporation,Released
6,Sep 8 2022 1:47PM,245982,10,0085969907,ISDIN Corporation,Released
7,Sep 8 2022 1:47PM,245982,10,0085969903,ISDIN Corporation,Released
8,Sep 8 2022 1:47PM,245982,10,0085969905,ISDIN Corporation,Released
9,Sep 8 2022 1:47PM,245982,10,0085969904,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,245979,Released,6
34,245980,Released,7
35,245981,Released,1
36,245982,Released,12
37,245983,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245979,NaN,NaN,6.0
245980,NaN,NaN,7.0
245981,NaN,NaN,1.0
245982,NaN,NaN,12.0
245983,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
245650,0.0,1.0,0.0
245758,1.0,0.0,0.0
245895,0.0,0.0,1.0
245898,0.0,0.0,4.0
245916,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
245650,0.0,1,0
245758,1.0,0,0
245895,0.0,0,1
245898,0.0,0,4
245916,0.0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,245650,0.0,1,0
1,245758,1.0,0,0
2,245895,0.0,0,1
3,245898,0.0,0,4
4,245916,0.0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,245650,0.0,1,
1,245758,1.0,,
2,245895,0.0,,1
3,245898,0.0,,4
4,245916,0.0,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC"
1,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation
13,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated"
14,Sep 8 2022 1:16PM,245980,10,"Snap Medical Industries, LLC"
21,Sep 8 2022 1:13PM,245979,10,Ivaoes Animal Health
27,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC
37,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions
38,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation
39,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC"
40,Sep 8 2022 12:32PM,245971,10,Beach Products Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",0.0,,1
1,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,0.0,,12
2,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",0.0,,1
3,Sep 8 2022 1:16PM,245980,10,"Snap Medical Industries, LLC",0.0,,7
4,Sep 8 2022 1:13PM,245979,10,Ivaoes Animal Health,0.0,,6
5,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,0.0,,10
6,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,0.0,,1
7,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,0.0,,1
8,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",0.0,,1
9,Sep 8 2022 12:32PM,245971,10,Beach Products Inc,0.0,4,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",1,
1,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,12,
2,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",1,
3,Sep 8 2022 1:16PM,245980,10,"Snap Medical Industries, LLC",7,
4,Sep 8 2022 1:13PM,245979,10,Ivaoes Animal Health,6,
5,Sep 8 2022 12:43PM,245976,10,MedStone Pharma LLC,10,
6,Sep 8 2022 12:42PM,245975,16,Sartorius Bioprocess Solutions,1,
7,Sep 8 2022 12:41PM,245974,10,ISDIN Corporation,1,
8,Sep 8 2022 12:39PM,245973,10,"Snap Medical Industries, LLC",1,
9,Sep 8 2022 12:32PM,245971,10,Beach Products Inc,12,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",1,
1,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,12,
2,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",1,
3,Sep 8 2022 1:16PM,245980,10,"Snap Medical Industries, LLC",7,
4,Sep 8 2022 1:13PM,245979,10,Ivaoes Animal Health,6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",1.0,NaN
1,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,12.0,NaN
2,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",1.0,NaN
3,Sep 8 2022 1:16PM,245980,10,"Snap Medical Industries, LLC",7.0,NaN
4,Sep 8 2022 1:13PM,245979,10,Ivaoes Animal Health,6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 8 2022 1:48PM,245983,16,"Emersa Waterbox, LLC",1.0,0.0
1,Sep 8 2022 1:47PM,245982,10,ISDIN Corporation,12.0,0.0
2,Sep 8 2022 1:37PM,245981,10,"Senseonics, Incorporated",1.0,0.0
3,Sep 8 2022 1:16PM,245980,10,"Snap Medical Industries, LLC",7.0,0.0
4,Sep 8 2022 1:13PM,245979,10,Ivaoes Animal Health,6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4918888,146.0,23.0
15,491910,65.0,38.0
16,737856,6.0,0.0
18,245934,0.0,1.0
19,245916,1.0,0.0
20,491848,7.0,4.0
21,491606,0.0,2.0
22,491933,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4918888,146.0,23.0
1,15,491910,65.0,38.0
2,16,737856,6.0,0.0
3,18,245934,0.0,1.0
4,19,245916,1.0,0.0
5,20,491848,7.0,4.0
6,21,491606,0.0,2.0
7,22,491933,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,146.0,23.0
1,15,65.0,38.0
2,16,6.0,0.0
3,18,0.0,1.0
4,19,1.0,0.0
5,20,7.0,4.0
6,21,0.0,2.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,146.0
1,15,Released,65.0
2,16,Released,6.0
3,18,Released,0.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Executing,23.0,38.0,0.0,1.0,0.0,4.0,2.0,0.0
Released,146.0,65.0,6.0,0.0,1.0,7.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Executing,23.0,38.0,0.0,1.0,0.0,4.0,2.0,0.0
1,Released,146.0,65.0,6.0,0.0,1.0,7.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Executing,23.0,38.0,0.0,1.0,0.0,4.0,2.0,0.0
1,Released,146.0,65.0,6.0,0.0,1.0,7.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()